In [ ]:
!pip install matplotlib
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install numpy

In [ ]:
import torchvision.models as models

# Load a pre-trained ResNet-34
resnet18 = models.resnet18(pretrained=True)

# Load an untrained ResNet-34
resnet18_untrained = models.resnet18(pretrained=False)

print(resnet18)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


In [ ]:
class CNNWith5HiddenLayers(nn.Module):
    def __init__(self):
        super(CNNWith5HiddenLayers, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)     # Hidden layer 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    # Hidden layer 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)   # Hidden layer 3
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  # Hidden layer 4
        self.pool = nn.MaxPool2d(2, 2)                  # Auxiliary layer (Pooling)
        self.fc1 = nn.Linear(256 * 2 * 2, 512)          # Hidden layer 5 (Fully connected)
        self.fc2 = nn.Linear(512, 10)                   # Output layer
        self.relu = nn.ReLU()                           # Activation function
        self.dropout = nn.Dropout(0.5)                  # Auxiliary layer (Dropout)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
def train_model(model, trainloader, criterion, optimizer, device, epochs=25):
    train_loss = []
    train_accuracy = []
    
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            if i % 100 == 99:  # Print every 100 batches
                print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}")
        
        accuracy = 100 * correct / total
        train_loss.append(running_loss / len(trainloader))
        train_accuracy.append(accuracy)
        print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%")
    
    return train_loss, train_accuracy


In [ ]:
def evaluate_model(model, testloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    all_labels = []
    all_outputs = []
    with torch.inference_mode():
        for i, (inputs, labels) in enumerate(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
    
    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)
    
    # Compute AUC for each class and average them
    auc_scores = []
    for i in range(10):  # Assuming 10 classes for CIFAR-10
        auc = roc_auc_score(all_labels == i, all_outputs[:, i])
        auc_scores.append(auc)
    
    mean_auc = np.mean(auc_scores)
    accuracy = 100 * correct / total
    print(f"AUC scores for each class: {auc_scores}")
    print(f"Accuracy: {accuracy:.2f}%")
    return mean_auc


In [ ]:
model = CNNWith5HiddenLayers().to(device)


# Load the CIFAR-10 dataset

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Train the model
train_loss, train_accuracy = train_model(model, trainloader, criterion, optimizer, device, epochs=25)


In [ ]:
# Evaluate the model
auc = evaluate_model(model, testloader, device)
print(f"Mean AUC: {auc}")

# Plot the training loss and accuracy
epochs = range(1, 26)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
